In [1]:
# imports
import os
import time
import datetime
import csv
import re

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch, helpers
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed

In [2]:
# Elastic host
es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    http_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
    verify_certs=False,
#     ca_certs="./ca.crt"
    timeout=30, 
    max_retries=10, 
    retry_on_timeout=True
)

# dir
translated_csv_dir = r"D:\Debate_a_Base\Data\Parlamint_3.0\Csv_set"

In [3]:
# it is VERY dangerous to uncomment this
# es.indices.delete(index='ngrams1')
# es.indices.delete(index='ngrams2')
# es.indices.delete(index='ngrams3')
# es.indices.delete(index='ngrams4')
# es.indices.delete(index='ngrams5')

In [4]:
# return list met dict [{Land1}, {Land 1}]
def get_csv_files(country_selection):
    os.chdir(translated_csv_dir)
    
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():
        
        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:
                    
                    file_data.append(file)

                paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [5]:
# haal jaar maand en dag uit filename
def extract_file_date(file_name):
    
    year_month_day = re.search(r"\d{4}-\d{2}-\d{2}", file_name)
#     year, month, day = year_month_day[0].split("-")
    
    return year_month_day[0]

### Shingles uit CSV

In [6]:
# geeft een dict met dates als keys en files als vals
def get_files_per_date(country, after=None):
    files = get_csv_files(country)[0][country]
    
    dates_dict = {}
    
    if after is not None:
        
        after = time.mktime(datetime.datetime.strptime(after, "%Y-%m-%d").timetuple())
    
    # loop door alle files heen
    for file in files:
        
        date = extract_file_date(file)
        
        # filter al geuploade dates uit de dict
        if after is not None:
            timestamp = time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())
            
            if (timestamp - after) <= 0:
                
                continue
        
        if date in dates_dict.keys():
            
            dates_dict[date] += [file]
            
        else:
            
            dates_dict[date] = [file]
            
    return dates_dict

In [7]:
# maakt paths aan voor alle files die bij een date horen
def get_date_paths(root, files):
    os.chdir(translated_csv_dir)
    
    paths = []
    
    for file in files:
        paths.append(os.path.join(root, file))
        
    return paths

In [8]:
# maakt ngrams van alle tekst in een csv file
def get_date_vocabulary(paths):
    os.chdir(translated_csv_dir)
    
    vocabulary = np.array(())
    
    for path in paths:
        df = pd.read_csv(path)
    
        # slecht vertaalde notebooks met alleen een index negeren
        if len(df.index) == 0:
            continue
            
        lines = [x for x in df["value"].to_list() if str(x) != 'nan']
    
        victor = CountVectorizer(ngram_range=(1, 5), token_pattern=r"(?u)\b\w+\b")
        victor.fit_transform(lines)
        
        vocabulary = np.unique(np.hstack((vocabulary, victor.get_feature_names_out()))) 
        
    return vocabulary

In [9]:
def get_date_counts(paths, vocabulary):
    os.chdir(translated_csv_dir)
    
    counts = np.zeros(shape=(len(vocabulary)), dtype=int)
    
    for path in paths:
        df = pd.read_csv(path)
    
        # slecht vertaalde notebooks met alleen een index negeren
        if len(df.index) == 0:
            continue
            
        lines = [x for x in df["value"].to_list() if str(x) != 'nan']
    
        victor = CountVectorizer(ngram_range=(1, 5), token_pattern=r"(?u)\b\w+\b")
        victor.fit_transform(vocabulary)
        
        counts = counts + np.sum(victor.transform(lines).toarray(), axis=0)
    
    return counts

In [10]:
def get_es_date_dict(vocabulary, country, date, counts, index="ngrams"):
    year, month, day = re.search(r"\d{4}-\d{2}-\d{2}", date)[0].split("-")
    
    bulk = []
    
    for ngram, count in zip(vocabulary, counts):
        content_dict, es_content_dict = {}, {}
        
        # entry inhoud
        content_dict["ngram"] = ngram
        content_dict["country"] = country
        content_dict["year"] = year
        content_dict["month"] = month
        content_dict["day"] = day
        content_dict["count"] = count
        
        # entry technische dingen
        es_content_dict["_index"] = f"{index}{len(ngram.split(' '))}"
        es_content_dict["_source"] = content_dict
        
        # voeg entry toe aan bulk return lijst
        bulk.append(es_content_dict)
        
    return bulk

In [11]:
# # functie testruimte
# t_folder = "BE"
# t_date = '2014-11-18'

# test_dates_dict = get_files_per_date(t_folder)
# print(test_dates_dict)
# test_paths = get_date_paths(t_folder, test_dates_dict[t_date ])
# print(test_paths)
# test_vocabulary = get_date_vocabulary(test_paths)
# print(test_vocabulary)
# test_counts = get_date_counts(test_paths, test_vocabulary)
# print(test_counts)
# test_date_es_dict = get_es_date_dict(test_vocabulary, t_folder, t_date, test_counts, 0)
# print(test_date_es_dict)

### Error management

In [12]:
log_loc = os.path.join(f"D:\Debate_a_Base\Data\Parlamint_3.0", "uploaded_ngram_dates.csv")

# houd bij welke files zijn geupload tijdens multiprocessing
def log_date_upload(country, date):
    with open(log_loc, 'a', newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow([country, date])

In [13]:
# filter de dates die al indexed zijn
def get_remaining_dates(country, dates):
    df_processed_dates = pd.read_csv(os.path.join(f"D:\Debate_a_Base\Data\Parlamint_3.0", "uploaded_ngram_dates.csv"))
    
    df_processed_dates_cutout = df_processed_dates[df_processed_dates["key"] == country]
    
    if len(df_processed_dates_cutout) != 0:
        disposable_dates = list(df_processed_dates_cutout["value"])
        
        for disposable_date in disposable_dates:
            if disposable_date in dates.keys():
                dates.pop(disposable_date)
                
    return dates

### Countvectorizer 1.3
Elastic (index = ngrams):
- | Shingle | Land | Jaar | Maand | Dag | Aantal | Percentage |
 - Shingles doen voor een datum, voor een land
 - Percentage berekenen
 - Uploaden in ES

In [14]:
def upload_ngrams(countries):
    
    for country in countries:
        
        # vraag de dates op die geprocessed moeten worden
        dates = get_remaining_dates(country, get_files_per_date(country))
        if len(dates) == 0:
            continue
            
        # dingen voor overzicht in prints
        timer = time.time()
        total_dates = len(dates.keys())
        
        def multithread_dates(date):        
            paths = get_date_paths(country, dates[date])
            
            vocabulary = get_date_vocabulary(paths)
            counts = get_date_counts(paths, vocabulary)
            
            elastic_dict = get_es_date_dict(vocabulary, country, date, counts)
            
            helpers.bulk(es, elastic_dict)
            log_date_upload(country, date)
            print(f"[Info]: {round((time.time() - timer) / 60, 2)}m, Uploaded: {date} {country}")
            
        # process multiple dates at the same time
        Parallel(n_jobs=15, prefer="threads")(delayed(multithread_dates)(date) for date in dates)
            
        print(f"[Info]: {country} finished")
        
    return True

In [15]:
# werkende landen
todo = ['AT', 'BA', 'BE', 'BG', 'CZ', 'DK', 'EE', 'ES-CT', 'ES-GA', 'FR', 'GR', 'HR', 'HU', 'IS', 'IT', 'LV', 'NL', 'NO', 'PL', 'PT', 'RS', 'SE', 'SI', 'TR', 'UA', 'GB']

done = []

upload = upload_ngrams(todo)

print(upload)

[Info]: 1.15m, Uploaded: 1999-11-10 RS
[Info]: 1.32m, Uploaded: 2003-02-06 RS
[Info]: 1.36m, Uploaded: 2012-02-29 RS
[Info]: 1.52m, Uploaded: 2004-02-20 RS
[Info]: 1.66m, Uploaded: 2016-01-19 RS
[Info]: 1.7m, Uploaded: 2006-04-03 RS
[Info]: 1.83m, Uploaded: 2019-03-05 RS
[Info]: 1.89m, Uploaded: 1999-11-11 RS
[Info]: 1.94m, Uploaded: 2014-04-26 RS
[Info]: 1.96m, Uploaded: 2010-02-25 RS
[Info]: 2.16m, Uploaded: 2017-04-20 RS
[Info]: 2.17m, Uploaded: 1998-05-26 RS
[Info]: 2.27m, Uploaded: 2020-01-21 RS
[Info]: 2.32m, Uploaded: 2000-10-24 RS
[Info]: 2.47m, Uploaded: 2000-10-23 RS
[Info]: 2.52m, Uploaded: 2012-02-27 RS
[Info]: 2.57m, Uploaded: 2005-01-26 RS
[Info]: 2.72m, Uploaded: 2002-01-29 RS
[Info]: 2.76m, Uploaded: 2013-01-22 RS
[Info]: 2.99m, Uploaded: 1998-05-27 RS
[Info]: 3.02m, Uploaded: 2015-01-19 RS
[Info]: 3.12m, Uploaded: 2008-06-30 RS
[Info]: 3.25m, Uploaded: 2002-01-30 RS
[Info]: 3.27m, Uploaded: 2003-02-11 RS
[Info]: 3.27m, Uploaded: 2011-02-23 RS
[Info]: 3.52m, Uploaded: 2

[Info]: 23.17m, Uploaded: 2008-07-21 RS
[Info]: 23.25m, Uploaded: 2005-02-16 RS
[Info]: 23.27m, Uploaded: 2014-05-22 RS
[Info]: 23.38m, Uploaded: 2001-03-29 RS
[Info]: 23.41m, Uploaded: 2014-05-23 RS
[Info]: 23.45m, Uploaded: 2015-02-02 RS
[Info]: 23.48m, Uploaded: 2007-05-29 RS
[Info]: 23.49m, Uploaded: 2009-02-02 RS
[Info]: 23.58m, Uploaded: 2019-04-03 RS
[Info]: 23.87m, Uploaded: 2002-02-21 RS
[Info]: 23.89m, Uploaded: 2008-07-18 RS
[Info]: 24.04m, Uploaded: 2010-03-12 RS
[Info]: 24.09m, Uploaded: 2003-03-18 RS
[Info]: 24.2m, Uploaded: 2007-05-28 RS
[Info]: 24.27m, Uploaded: 2018-03-26 RS
[Info]: 24.29m, Uploaded: 2019-04-02 RS
[Info]: 24.37m, Uploaded: 2017-05-19 RS
[Info]: 24.46m, Uploaded: 2006-04-13 RS
[Info]: 24.51m, Uploaded: 2009-01-29 RS
[Info]: 24.63m, Uploaded: 2013-02-15 RS
[Info]: 24.68m, Uploaded: 2012-08-06 RS
[Info]: 24.77m, Uploaded: 2010-03-11 RS
[Info]: 24.89m, Uploaded: 2005-02-18 RS
[Info]: 24.96m, Uploaded: 2020-01-30 RS
[Info]: 24.99m, Uploaded: 2020-01-31 RS
[

[Info]: 42.23m, Uploaded: 2020-02-18 RS
[Info]: 42.28m, Uploaded: 2002-04-15 RS
[Info]: 42.37m, Uploaded: 2009-03-10 RS
[Info]: 42.43m, Uploaded: 2021-03-18 RS
[Info]: 42.53m, Uploaded: 2017-07-17 RS
[Info]: 42.55m, Uploaded: 2004-05-12 RS
[Info]: 42.65m, Uploaded: 2019-05-21 RS
[Info]: 42.93m, Uploaded: 2007-06-20 RS
[Info]: 42.97m, Uploaded: 2013-03-13 RS
[Info]: 42.98m, Uploaded: 2005-04-12 RS
[Info]: 43.12m, Uploaded: 2010-03-30 RS
[Info]: 43.23m, Uploaded: 2014-06-12 RS
[Info]: 43.26m, Uploaded: 2002-04-16 RS
[Info]: 43.55m, Uploaded: 2003-04-15 RS
[Info]: 43.61m, Uploaded: 2001-04-23 RS
[Info]: 43.63m, Uploaded: 2008-09-29 RS
[Info]: 43.75m, Uploaded: 2019-05-20 RS
[Info]: 43.87m, Uploaded: 2015-03-10 RS
[Info]: 44.12m, Uploaded: 2010-03-31 RS
[Info]: 44.16m, Uploaded: 2017-06-29 RS
[Info]: 44.25m, Uploaded: 2004-05-13 RS
[Info]: 44.25m, Uploaded: 2012-09-22 RS
[Info]: 44.26m, Uploaded: 2020-02-19 RS
[Info]: 44.46m, Uploaded: 2007-06-21 RS
[Info]: 44.57m, Uploaded: 2001-04-24 RS


[Info]: 64.72m, Uploaded: 2020-02-28 RS
[Info]: 64.97m, Uploaded: 2009-03-26 RS
[Info]: 65.0m, Uploaded: 2009-03-25 RS
[Info]: 65.36m, Uploaded: 2011-04-27 RS
[Info]: 65.38m, Uploaded: 2008-10-16 RS
[Info]: 65.73m, Uploaded: 2001-05-15 RS
[Info]: 65.76m, Uploaded: 2012-10-31 RS
[Info]: 65.77m, Uploaded: 2019-06-19 RS
[Info]: 65.89m, Uploaded: 2013-04-08 RS
[Info]: 65.9m, Uploaded: 2003-05-27 RS
[Info]: 65.95m, Uploaded: 2013-04-05 RS
[Info]: 66.03m, Uploaded: 2012-10-30 RS
[Info]: 66.07m, Uploaded: 2021-04-07 RS
[Info]: 66.13m, Uploaded: 2019-06-21 RS
[Info]: 66.21m, Uploaded: 2007-07-17 RS
[Info]: 66.36m, Uploaded: 2011-05-05 RS
[Info]: 66.42m, Uploaded: 2017-09-25 RS
[Info]: 66.55m, Uploaded: 2004-07-06 RS
[Info]: 66.56m, Uploaded: 2009-03-30 RS
[Info]: 66.62m, Uploaded: 2009-03-31 RS
[Info]: 66.65m, Uploaded: 2015-04-09 RS
[Info]: 66.86m, Uploaded: 2010-04-15 RS
[Info]: 66.99m, Uploaded: 2011-04-28 RS
[Info]: 67.04m, Uploaded: 2020-02-29 RS
[Info]: 67.22m, Uploaded: 2009-04-01 RS
[I

[Info]: 86.88m, Uploaded: 2011-05-25 RS
[Info]: 87.17m, Uploaded: 2014-08-01 RS
[Info]: 87.2m, Uploaded: 2017-10-12 RS
[Info]: 87.44m, Uploaded: 2004-08-17 RS
[Info]: 87.46m, Uploaded: 2016-08-11 RS
[Info]: 87.6m, Uploaded: 2005-06-29 RS
[Info]: 87.62m, Uploaded: 2015-05-08 RS
[Info]: 87.72m, Uploaded: 2003-06-24 RS
[Info]: 87.79m, Uploaded: 2015-05-07 RS
[Info]: 87.83m, Uploaded: 2004-07-22 RS
[Info]: 87.99m, Uploaded: 2020-05-08 RS
[Info]: 88.25m, Uploaded: 2021-04-27 RS
[Info]: 88.33m, Uploaded: 2008-10-29 RS
[Info]: 88.34m, Uploaded: 2013-05-21 RS
[Info]: 88.43m, Uploaded: 2007-09-17 RS
[Info]: 88.51m, Uploaded: 2009-04-15 RS
[Info]: 88.53m, Uploaded: 2004-08-18 RS
[Info]: 88.57m, Uploaded: 2013-05-20 RS
[Info]: 88.67m, Uploaded: 2016-08-12 RS
[Info]: 88.87m, Uploaded: 2013-05-22 RS
[Info]: 89.33m, Uploaded: 2019-07-04 RS
[Info]: 89.41m, Uploaded: 2019-07-08 RS
[Info]: 89.51m, Uploaded: 2003-06-25 RS
[Info]: 89.58m, Uploaded: 2020-08-03 RS
[Info]: 89.71m, Uploaded: 2010-05-11 RS
[I

...

[Info]: 85.5m, Uploaded: 2016-07-14 GB
[Info]: 86.4m, Uploaded: 2016-07-21 GB
[Info]: 86.44m, Uploaded: 2016-09-06 GB
[Info]: 86.56m, Uploaded: 2016-09-05 GB
[Info]: 87.33m, Uploaded: 2016-09-07 GB
[Info]: 88.28m, Uploaded: 2016-09-12 GB
[Info]: 88.48m, Uploaded: 2016-09-08 GB
[Info]: 88.94m, Uploaded: 2016-09-13 GB
[Info]: 89.17m, Uploaded: 2016-09-14 GB
[Info]: 89.5m, Uploaded: 2016-10-17 GB
[Info]: 89.61m, Uploaded: 2016-10-21 GB
[Info]: 89.78m, Uploaded: 2016-10-11 GB
[Info]: 90.09m, Uploaded: 2016-09-15 GB
[Info]: 90.52m, Uploaded: 2016-10-10 GB
[Info]: 90.89m, Uploaded: 2016-10-19 GB
[Info]: 91.01m, Uploaded: 2016-10-18 GB
[Info]: 91.21m, Uploaded: 2016-10-12 GB
[Info]: 91.51m, Uploaded: 2016-10-13 GB
[Info]: 91.68m, Uploaded: 2016-10-28 GB
[Info]: 91.69m, Uploaded: 2016-10-20 GB
[Info]: 92.0m, Uploaded: 2016-10-24 GB
[Info]: 92.73m, Uploaded: 2016-11-04 GB
[Info]: 93.48m, Uploaded: 2016-10-25 GB
[Info]: 93.57m, Uploaded: 2016-11-14 GB
[Info]: 93.92m, Uploaded: 2016-11-09 GB
[Inf

[Info]: 152.79m, Uploaded: 2017-12-19 GB
[Info]: 153.62m, Uploaded: 2018-01-15 GB
[Info]: 153.85m, Uploaded: 2018-01-10 GB
[Info]: 154.33m, Uploaded: 2018-01-26 GB
[Info]: 154.45m, Uploaded: 2018-01-17 GB
[Info]: 154.97m, Uploaded: 2018-01-16 GB
[Info]: 155.45m, Uploaded: 2018-01-23 GB
[Info]: 155.46m, Uploaded: 2018-01-11 GB
[Info]: 155.47m, Uploaded: 2018-01-22 GB
[Info]: 156.22m, Uploaded: 2018-02-02 GB
[Info]: 156.72m, Uploaded: 2018-01-18 GB
[Info]: 157.32m, Uploaded: 2018-01-24 GB
[Info]: 158.09m, Uploaded: 2018-01-29 GB
[Info]: 158.35m, Uploaded: 2018-02-01 GB
[Info]: 158.45m, Uploaded: 2018-01-25 GB
[Info]: 159.28m, Uploaded: 2018-02-07 GB
[Info]: 159.71m, Uploaded: 2018-02-05 GB
[Info]: 159.89m, Uploaded: 2018-02-06 GB
[Info]: 159.97m, Uploaded: 2018-01-30 GB
[Info]: 160.03m, Uploaded: 2018-02-23 GB
[Info]: 160.34m, Uploaded: 2018-02-08 GB
[Info]: 160.68m, Uploaded: 2018-01-31 GB
[Info]: 161.42m, Uploaded: 2018-02-22 GB
[Info]: 161.64m, Uploaded: 2018-02-21 GB
[Info]: 162.0m, 

[Info]: 215.84m, Uploaded: 2019-03-19 GB
[Info]: 215.84m, Uploaded: 2019-03-14 GB
[Info]: 216.26m, Uploaded: 2019-03-11 GB
[Info]: 216.49m, Uploaded: 2019-03-13 GB
[Info]: 216.49m, Uploaded: 2019-03-12 GB
[Info]: 217.01m, Uploaded: 2019-03-18 GB
[Info]: 217.09m, Uploaded: 2019-03-29 GB
[Info]: 217.79m, Uploaded: 2019-03-20 GB
[Info]: 217.86m, Uploaded: 2019-03-21 GB
[Info]: 218.23m, Uploaded: 2019-03-26 GB
[Info]: 218.62m, Uploaded: 2019-03-25 GB
[Info]: 218.91m, Uploaded: 2019-03-27 GB
[Info]: 219.25m, Uploaded: 2019-04-02 GB
[Info]: 219.75m, Uploaded: 2019-04-01 GB
[Info]: 219.78m, Uploaded: 2019-04-23 GB
[Info]: 219.97m, Uploaded: 2019-04-10 GB
[Info]: 220.22m, Uploaded: 2019-04-09 GB
[Info]: 220.43m, Uploaded: 2019-03-28 GB
[Info]: 220.46m, Uploaded: 2019-04-08 GB
[Info]: 220.82m, Uploaded: 2019-04-04 GB
[Info]: 220.96m, Uploaded: 2019-04-03 GB
[Info]: 221.58m, Uploaded: 2019-04-11 GB
[Info]: 222.33m, Uploaded: 2019-04-29 GB
[Info]: 222.8m, Uploaded: 2019-04-30 GB
[Info]: 222.93m, 

[Info]: 275.59m, Uploaded: 2020-09-01 GB
[Info]: 276.0m, Uploaded: 2020-09-11 GB
[Info]: 276.19m, Uploaded: 2020-07-21 GB
[Info]: 276.32m, Uploaded: 2020-09-03 GB
[Info]: 276.44m, Uploaded: 2020-07-22 GB
[Info]: 276.6m, Uploaded: 2020-09-18 GB
[Info]: 277.51m, Uploaded: 2020-09-02 GB
[Info]: 277.85m, Uploaded: 2020-09-07 GB
[Info]: 278.16m, Uploaded: 2020-09-08 GB
[Info]: 278.64m, Uploaded: 2020-09-10 GB
[Info]: 279.39m, Uploaded: 2020-09-15 GB
[Info]: 279.4m, Uploaded: 2020-09-09 GB
[Info]: 279.52m, Uploaded: 2020-09-25 GB
[Info]: 280.14m, Uploaded: 2020-09-14 GB
[Info]: 280.17m, Uploaded: 2020-09-16 GB
[Info]: 280.36m, Uploaded: 2020-09-21 GB
[Info]: 280.52m, Uploaded: 2020-09-24 GB
[Info]: 281.11m, Uploaded: 2020-09-17 GB
[Info]: 281.3m, Uploaded: 2020-10-09 GB
[Info]: 281.34m, Uploaded: 2020-09-22 GB
[Info]: 281.79m, Uploaded: 2020-09-23 GB
[Info]: 281.79m, Uploaded: 2020-09-29 GB
[Info]: 281.84m, Uploaded: 2020-09-28 GB
[Info]: 282.63m, Uploaded: 2020-10-01 GB
[Info]: 283.09m, Upl

[Info]: 338.49m, Uploaded: 2021-11-22 GB
[Info]: 338.51m, Uploaded: 2021-11-30 GB
[Info]: 339.25m, Uploaded: 2021-11-25 GB
[Info]: 339.45m, Uploaded: 2021-12-03 GB
[Info]: 340.54m, Uploaded: 2021-12-10 GB
[Info]: 340.56m, Uploaded: 2021-12-02 GB
[Info]: 340.6m, Uploaded: 2021-12-06 GB
[Info]: 341.18m, Uploaded: 2021-12-08 GB
[Info]: 341.62m, Uploaded: 2021-12-13 GB
[Info]: 341.73m, Uploaded: 2021-12-07 GB
[Info]: 341.9m, Uploaded: 2021-12-09 GB
[Info]: 342.12m, Uploaded: 2021-12-15 GB
[Info]: 342.41m, Uploaded: 2022-01-14 GB
[Info]: 342.53m, Uploaded: 2022-01-12 GB
[Info]: 342.94m, Uploaded: 2021-12-14 GB
[Info]: 343.38m, Uploaded: 2022-01-05 GB
[Info]: 343.6m, Uploaded: 2021-12-16 GB
[Info]: 343.69m, Uploaded: 2022-01-10 GB
[Info]: 343.91m, Uploaded: 2022-01-06 GB
[Info]: 344.05m, Uploaded: 2022-01-21 GB
[Info]: 344.31m, Uploaded: 2022-01-11 GB
[Info]: 345.45m, Uploaded: 2022-01-17 GB
[Info]: 345.81m, Uploaded: 2022-01-13 GB
[Info]: 346.26m, Uploaded: 2022-01-18 GB
[Info]: 346.61m, Up